In [1]:
import brightway2 as bw
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python
import csv
import stats_arrays
import scipy as sp
import pandas as pd
import pickle
import h5py
import time

In [2]:
bw.projects.set_current('iw_integration')

# Save and load functions with pickle

In [3]:
def pickle_save(file_path_root,object_name_to_save,file_name):
    
    complete_file_path=file_path_root+'\\'+file_name+'.p'
    
    pickle.dump( object_name_to_save, open( complete_file_path, "wb" ) )
    
    return;




def pickle_load(file_path_root,file_name):
    
    complete_file_path=file_path_root+'\\'+file_name+'.p'
    
    return pickle.load( open( complete_file_path, "rb" ) );
    


In [ ]:
#file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
#object_name_to_save=object_to_save
#file_name='object_to_save'

pickle_save(file_path_root,object_name_to_save,file_name)

object_to_load=pickle_load(file_path_root,file_name)

# Save and load LCA object for an activity in eiv33

Pick a random process in eiv33 and construct an LCA object

In [22]:
DB_eiv33=bw.Database('ecoinvent 3.3 cutoff')

In [65]:
act_random_eiv33=DB_eiv33.random()

In [4]:
def create_LCA_object(act,fu,ic_name=None):
    LCA_object=bw.LCA({act:fu},ic_name)
    return LCA_object;

In [67]:
act=act_random_eiv33
fu=1

LCA_object=create_LCA_object(act,fu,ic_name=None)

Create A and B matrices with .load_lci_data() and save the resulting objects with pickle

In [5]:
def create_A_B(LCA_obj,params_only=False, matrix_only=False):
    LCA_obj.load_lci_data(fix_dictionaries=False)
    dict_load_lci_data_objects={}
    
    if params_only==True:
        dict_load_lci_data_objects={'bio_params':LCA_obj.bio_params,
                               'tech_params':LCA_obj.tech_params}
    elif matrix_only==True:
        dict_load_lci_data_objects={'biosphere_matrix':LCA_obj.biosphere_matrix,
                               'technosphere_matrix':LCA_obj.technosphere_matrix}
    
    else:
        dict_load_lci_data_objects={'bio_params':LCA_obj.bio_params,
                               'tech_params':LCA_obj.tech_params,
                               'biosphere_dict':LCA_obj.biosphere_dict,
                               'activity_dict':LCA_obj.activity_dict,
                               'product_dict':LCA_obj.product_dict,
                               'biosphere_matrix':LCA_obj.biosphere_matrix,
                               'technosphere_matrix':LCA_obj.technosphere_matrix}
    
    return dict_load_lci_data_objects;

In [69]:
dict_A_B=create_A_B(LCA_object)

In [70]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=dict_A_B
file_name='dict_A_B_random_eiv33'

pickle_save(file_path_root,object_name_to_save,file_name)

Import the LCA object and finish the calculation

In [71]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=dict_A_B
file_name='dict_A_B_random_eiv33'

dict_A_B_load=pickle_load(file_path_root,file_name)

In [6]:
def load_A_B_to_LCA_object(dict_A_B,act,fu,ic_name=None):
    
    LCA_obj=create_LCA_object(act,fu,ic_name)
    
    #This function recreates the objects created during the .load_lci_data()
    
    #to replace builder.build(self.database_filepath) in .load_lci_data()
    LCA_obj.bio_params=dict_A_B['bio_params']
    LCA_obj.tech_params=dict_A_B['tech_params']
    LCA_obj.biosphere_dict=dict_A_B['biosphere_dict']
    LCA_obj.activity_dict=dict_A_B['activity_dict']
    LCA_obj.product_dict=dict_A_B['product_dict']
    LCA_obj.biosphere_matrix=dict_A_B['biosphere_matrix']
    LCA_obj.technosphere_matrix=dict_A_B['technosphere_matrix']

    LCA_obj.fix_dictionaries() #function in .load_lci_data()
    
    return LCA_obj;

In [73]:
ic_name=('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017','Ecosystem Quality','Land transformation, biodiversity, GLO, with uncert')

LCA_object_load=load_A_B_to_LCA_object(dict_A_B_load,act,fu,ic_name)

In [7]:
def finish_LCA_object(LCA_obj):

    #The beginning of .lci() function replace by the load_A_B_to_LCA_object function
    
    LCA_obj.build_demand_array() #function in .lci() after calling .load_lci_data()
    LCA_obj.lci_calculation() #function in .lci() after calling .load_lci_data()
    LCA_obj.lcia()
    
    return;

In [75]:
finish_LCA_object(LCA_object_load)

In [76]:
LCA_object_load.score

-0.00032472068199512565

## Comparing all created objects in LCA object to test if everything needs to be stored

In [51]:
act_random_eiv33_1=DB_eiv33.random()
act_random_eiv33_2=DB_eiv33.random()
act_1=act_random_eiv33_2
act_2=act_random_eiv33_2
fu=1

LCA_object_1=create_LCA_object(act_1,fu,ic_name=None)
LCA_object_2=create_LCA_object(act_2,fu,ic_name=None)
dict_A_B_1=create_A_B(LCA_object_1)
dict_A_B_2=create_A_B(LCA_object_2)


In [53]:
(dict_A_B_1['bio_params']!=dict_A_B_2['bio_params'],
 dict_A_B_1['tech_params']!=dict_A_B_2['tech_params'],
 dict_A_B_1['biosphere_dict']!=dict_A_B_2['biosphere_dict'],
 dict_A_B_1['activity_dict']!=dict_A_B_2['activity_dict'],
 dict_A_B_1['product_dict']!=dict_A_B_2['product_dict'],
 dict_A_B_1['biosphere_matrix']!=dict_A_B_2['biosphere_matrix'],
 dict_A_B_1['technosphere_matrix']!=dict_A_B_2['technosphere_matrix'])

(array([ True,  True,  True, ...,  True,  True,  True], dtype=bool),
 array([ True,  True,  True, ...,  True,  True,  True], dtype=bool),
 False,
 False,
 False,
 <1960x13831 sparse matrix of type '<class 'numpy.bool_'>'
 	with 0 stored elements in Compressed Sparse Row format>,
 <13831x13831 sparse matrix of type '<class 'numpy.bool_'>'
 	with 0 stored elements in Compressed Sparse Row format>)

In [61]:
biosphere_mat_comparison=LCA_object_1.biosphere_matrix.data!=LCA_object_2.biosphere_matrix.data
technosphere_mat_comparison=LCA_object_1.technosphere_matrix.data!=LCA_object_2.technosphere_matrix.data

In [62]:
for comp in biosphere_mat_comparison:
    if comp==True:
        print('biosphere matrices are different')
        
for comp in technosphere_mat_comparison:
    if comp==True:
        print('biosphere matrices are different')

Only bio_params and tech_params need to be stored, the other are the same for all activities in eiv33! 
--> around 20MB instead of 30MB for all the objects per activity per iteration

In [63]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=dict_A_B_1['bio_params']
file_name='dict_A_B_1_bio_params'

pickle_save(file_path_root,object_name_to_save,file_name)

In [64]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=dict_A_B_1['tech_params']
file_name='dict_A_B_1_tech_params'

pickle_save(file_path_root,object_name_to_save,file_name)

Write a code to only generate bio_params and tech_params? Ça l'air compliqué...

# Monte Carlo for LCI for an activity + storage as dictionary

About objects needed to recreate an LCA object:
-*biosphere_dict, activity_dict, product_dict* are the same for the entire DB
-*bio_params, tech_params* remain the same for an activity whatever the MC iteration
-*biosphere_matrix, technosphere_matrix* change for every MC iteration

In [8]:
DB_eiv33=bw.Database('ecoinvent 3.3 cutoff')

In [9]:
act=DB_eiv33.random()
fu=1
iterations=10
matrix_only=True

start = time.time()
LCA_obj=create_LCA_object(act,fu,ic_name=None)
end = time.time()
print("Calcul en {} secondes".format(end - start))


start = time.time()
dict_A_B=create_A_B(LCA_obj)
end = time.time()
print("Calcul en {} secondes".format(end - start))

Calcul en 0.0010006427764892578 secondes
Calcul en 0.2746436595916748 secondes


In [10]:
act.as_dict()

{'activity': '978d0459-d542-4858-b4f5-20d579093dc9',
 'activity type': 'market activity',
 'authors': {'data entry': {'email': 'levova@ecoinvent.org',
   'name': 'Tereza Levova'},
  'data generator': {'email': 'levova@ecoinvent.org',
   'name': 'Tereza Levova'}},
 'classifications': [('EcoSpold01Categories', 'biomass/fuels'),
  ('ISIC rev.4 ecoinvent',
   '3520:Manufacture of gas; distribution of gaseous fuels through mains')],
 'code': 'f171dd5c543667e25ad52e80efb83f0c',
 'comment': '',
 'database': 'ecoinvent 3.3 cutoff',
 'filename': '978d0459-d542-4858-b4f5-20d579093dc9_795369c8-09eb-439f-9321-15c0d3be6a6f.spold',
 'flow': '795369c8-09eb-439f-9321-15c0d3be6a6f',
 'location': 'CH',
 'name': 'market for synthetic gas',
 'parameters': {},
 'production amount': 1.0,
 'reference product': 'synthetic gas',
 'type': 'process',
 'unit': 'cubic meter'}

In [11]:
def LCI_MonteCarlo_from_LCA_object(act,fu,LCA_obj,iterations,matrix_only=True,seed=None):
    
    LCA_obj.fix_dictionaries()
    
    dict_act_MC_LCA_object={}
    dict_act_MC_LCA_object['iterations']=iterations
    dict_act_MC_LCA_object['activity key']=act.key
    dict_act_MC_LCA_object['classifications']=act.as_dict()['classifications']
    dict_act_MC_LCA_object['functional unit']=fu
    #dict_act_MC_LCA_object['bio_params']=LCA_obj.bio_params
    #dict_act_MC_LCA_object['tech_params']=LCA_obj.tech_params
    dict_act_MC_LCA_object['Monte Carlo LCI results']={} 
    
    
    LCA_obj.tech_rng = stats_arrays.random.MCRandomNumberGenerator(LCA_obj.tech_params, seed=seed)
    LCA_obj.bio_rng = stats_arrays.random.MCRandomNumberGenerator(LCA_obj.bio_params, seed=seed)
    
    for it in range(iterations):
        LCA_obj.rebuild_technosphere_matrix(LCA_obj.tech_rng.next())
        LCA_obj.rebuild_biosphere_matrix(LCA_obj.bio_rng.next())
        
        dict_act_MC_LCA_object['Monte Carlo LCI results'][it]=create_A_B(LCA_obj,matrix_only=matrix_only)
        
    return dict_act_MC_LCA_object;    

In [12]:
from bw2calc.utils import extract_uncertainty_fields as euf

def LCI_MonteCarlo_from_LCA_object_params(act,fu,LCA_obj,iterations,matrix_only=True,seed=None):
    
    LCA_obj.fix_dictionaries()
    
    dict_act_MC_LCA_object={}
    dict_act_MC_LCA_object['iterations']=iterations
    dict_act_MC_LCA_object['activity key']=act.key
    dict_act_MC_LCA_object['classifications']=act.as_dict()['classifications']
    dict_act_MC_LCA_object['functional unit']=fu
    #dict_act_MC_LCA_object['bio_params']=LCA_obj.bio_params
    #dict_act_MC_LCA_object['tech_params']=LCA_obj.tech_params
    dict_act_MC_LCA_object['Monte Carlo LCI results']={}
    dict_act_MC_LCA_object['LCI params values']={}
    
    
    # In order to store LCI parameters values at each iteration
    # useful for sensitivity analysis (Inspired from class ParameterVectorLCA)
    positions = {
    "tech": (0, LCA_obj.tech_params.shape[0]),
    "bio": (
        LCA_obj.tech_params.shape[0],
        LCA_obj.tech_params.shape[0] + LCA_obj.bio_params.shape[0]
        )
    }

    # `euf` is extract_uncertainty_fields; needed because we are
    # "gluing" together arrays with different column numbers and labels
    params = (euf(LCA_obj.tech_params), euf(LCA_obj.bio_params))
    
    LCA_obj.positions = positions
    LCA_obj.params = np.hstack(params)
    LCA_obj.rng = stats_arrays.random.MCRandomNumberGenerator(LCA_obj.params, seed=seed)
    
    
    #LCA_obj.tech_rng = stats_arrays.random.MCRandomNumberGenerator(LCA_obj.tech_params, seed=seed)
    #LCA_obj.bio_rng = stats_arrays.random.MCRandomNumberGenerator(LCA_obj.bio_params, seed=seed)
    
    for it in range(iterations):
        
        LCA_obj.sample = (LCA_obj.rng.next()).copy()
        LCA_obj.tech_sample=LCA_obj.sample[LCA_obj.positions["tech"][0]:LCA_obj.positions["tech"][1]]
        LCA_obj.bio_sample=LCA_obj.sample[LCA_obj.positions["bio"][0]:LCA_obj.positions["bio"][1]]     
        
        LCA_obj.rebuild_technosphere_matrix(LCA_obj.tech_sample)
        LCA_obj.rebuild_biosphere_matrix(LCA_obj.bio_sample)
        
        dict_act_MC_LCA_object['Monte Carlo LCI results'][it]=create_A_B(LCA_obj,matrix_only=matrix_only)
        dict_act_MC_LCA_object['LCI params values'][it]={'tech_sample':LCA_obj.tech_sample,'bio_sample':LCA_obj.bio_sample}
        
    return dict_act_MC_LCA_object;    

In [13]:
start = time.time()
dict_act_MC_LCA_object_A_B_only_10it_1act=LCI_MonteCarlo_from_LCA_object(act,fu,LCA_obj,iterations,matrix_only,seed=None)
end = time.time()
print("Calcul en {} secondes".format(end - start))

start = time.time()
dict_act_MC_LCA_object_A_B_only_10it_1ct_params=LCI_MonteCarlo_from_LCA_object_params(act,fu,LCA_obj,iterations,matrix_only,seed=None)
end = time.time()
print("Calcul en {} secondes".format(end - start))

Calcul en 3.693621873855591 secondes
Calcul en 3.662595748901367 secondes


In [14]:
current_path_dir=os.getcwd()
path_for_saving=current_path_dir+"\\Saved objects"
path_for_saving

'C:\\Users\\Laure\\Brightway - Notebooks and code\\Monte-Carlo\\Saved objects'

In [15]:
file_path_root=path_for_saving
object_name_to_save=dict_act_MC_LCA_object_A_B_only_10it_1act
file_name='dict_act_MC_LCA_object_A_B_only_10it_1act'

start = time.time()
pickle_save(file_path_root,object_name_to_save,file_name)
end = time.time()
print("Calcul en {} secondes".format(end - start))

Calcul en 0.2821519374847412 secondes


In [16]:
file_path_root=path_for_saving
object_name_to_save=dict_act_MC_LCA_object_A_B_only_10it_1ct_params
file_name='dict_act_MC_LCA_object_A_B_only_10it_1ct_params'

start = time.time()
pickle_save(file_path_root,object_name_to_save,file_name)
end = time.time()
print("Calcul en {} secondes".format(end - start))

Calcul en 0.4655475616455078 secondes


In [18]:
dict_act_MC_LCA_object

{'Monte Carlo LCI results': {0: {'biosphere_matrix': <1960x13831 sparse matrix of type '<class 'numpy.float64'>'
   	with 353369 stored elements in Compressed Sparse Row format>,
   'technosphere_matrix': <13831x13831 sparse matrix of type '<class 'numpy.float64'>'
   	with 147427 stored elements in Compressed Sparse Row format>},
  1: {'biosphere_matrix': <1960x13831 sparse matrix of type '<class 'numpy.float64'>'
   	with 353369 stored elements in Compressed Sparse Row format>,
   'technosphere_matrix': <13831x13831 sparse matrix of type '<class 'numpy.float64'>'
   	with 147427 stored elements in Compressed Sparse Row format>},
  2: {'biosphere_matrix': <1960x13831 sparse matrix of type '<class 'numpy.float64'>'
   	with 353369 stored elements in Compressed Sparse Row format>,
   'technosphere_matrix': <13831x13831 sparse matrix of type '<class 'numpy.float64'>'
   	with 147427 stored elements in Compressed Sparse Row format>},
  3: {'biosphere_matrix': <1960x13831 sparse matrix of 

In [19]:
dict_act_MC_LCA_object_params

{'LCI params values': {0: {'bio_sample': array([  6.80218176e-05,   6.75584498e-06,   2.65238031e-07, ...,
            1.81643585e-04,   4.85339759e-03,   1.40998506e-03]),
   'tech_sample': array([ 1.        ,  1.        ,  1.        , ..., -0.99901086,
           0.95062798,  0.99158311])},
  1: {'bio_sample': array([  9.20649337e-05,   7.72893635e-06,   2.92778103e-07, ...,
            1.24551225e-04,   2.03781673e-03,   1.36074204e-03]),
   'tech_sample': array([ 1.        ,  1.        ,  1.        , ..., -0.99901086,
           0.95062798,  0.99158311])},
  2: {'bio_sample': array([  3.89510175e-05,   4.98103755e-06,   2.72218821e-07, ...,
            2.29476828e-04,   6.14485700e-03,   2.97253798e-03]),
   'tech_sample': array([ 1.        ,  1.        ,  1.        , ..., -0.99901086,
           0.95062798,  0.99158311])},
  3: {'bio_sample': array([  2.85315978e-05,   5.56493061e-06,   2.93496712e-07, ...,
            1.78617835e-04,   3.31160848e-03,   3.67061081e-03]),
   'tech

In [49]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=dict_act_MC_LCA_object
file_name='dict_act_MC_LCA_object_matrix_only'

dict_act_MC_LCA_object_load=pickle_load(file_path_root,file_name)

Store results with HDF5

## Storage using big data package 

HDF5 format: http://deusyss.developpez.com/tutoriels/Python/hdf5/


Same size in MB than with pickle

In [11]:
import h5py
import time
import numpy as np

# Création d'une matrice
mon_tableau = np.ones((10000, 10000))

# création du fichier hdf5
mon_fichier = h5py.File(r'C:\Users\Laure\Brightway - Notebook\Notebook - tutorial\IW+ integration\HDF5 object saved\mon_fichier.hdf5', 'a')
mon_dataset = mon_fichier.create_dataset("fichier de demo2", data=mon_tableau)
mon_fichier.flush()

# Maintenant les données numériques sont aisément accessible
start = time.time()
mon_dataset[:, 2] = 56 * mon_dataset[:, 3]
end = time.time()
print("10 000 données modifiées en {} secondes".format(end - start))

mon_fichier.close()

10 000 données modifiées en 3.2879414558410645 secondes


In [12]:
file_path_root=r'C:\Users\Laure\Brightway - Notebook\Notebook - tutorial\IW+ integration\HDF5 object saved'
object_name_to_save=mon_tableau
file_name='mon_fichier'

pickle_save(file_path_root,object_name_to_save,file_name)

In [13]:
mon_fichier = h5py.File(r'C:\Users\Laure\Brightway - Notebook\Notebook - tutorial\IW+ integration\HDF5 object saved\mon_fichier.hdf5', 'a')

In [23]:
dataset_from_h5py=mon_fichier.get("fichier de demo2")

Convert dataset in numpy array

In [28]:
array=np.array(dataset_from_h5py)

In [29]:
array

array([[  1.,   1.,  56., ...,   1.,   1.,   1.],
       [  1.,   1.,  56., ...,   1.,   1.,   1.],
       [  1.,   1.,  56., ...,   1.,   1.,   1.],
       ..., 
       [  1.,   1.,  56., ...,   1.,   1.,   1.],
       [  1.,   1.,  56., ...,   1.,   1.,   1.],
       [  1.,   1.,  56., ...,   1.,   1.,   1.]])

In [31]:
len(array), type(array)

(10000, numpy.ndarray)